In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from matplotlib import pyplot as plt
import sklearn
import sklearn.cluster
%matplotlib inline

In [2]:
train = pd.read_csv('../train.csv')

In [3]:
test = pd.read_csv('../test.csv')

In [4]:
train_and_test = pd.concat([train[['lon', 'lat']], test[['lon', 'lat']]])

In [ ]:
cluster_learn = []
lonlat = train_and_test.values
for lon, lat in lonlat:
    cluster_learn.append([float(lon), float(lat)])
cluster_learn = np.array(cluster_learn)

In [ ]:
clst = sklearn.cluster.KMeans(init='k-means++', random_state=1337, n_clusters=5000, n_jobs=16)
ids = clst.fit_predict(cluster_learn)
centers = clst.cluster_centers

In [ ]:
train_and_test['cluster_id'] = ids

In [ ]:
import pygeoplot as gp
from IPython.display import Image

In [ ]:
hm = pd.DataFrame({'lon' : centers[:, 0], 'lat' : centers[:, 1]})
map_center = list(hm[:10].ix[:, ['lat', 'lon']].mean())

In [ ]:
points_map = gp.Map()
gp.placemarks_from_df(points_map, hm, lat_col='lat', lng_col='lon')
points_map.set_state(map_center, 5)

In [ ]:
points_map.save_html('cluster_centers.html')

In [ ]:
train_and_test[:len(train)].to_csv("train_cluster_ids.csv")
train_and_test[len(train):].to_csv("test_cluster_ids.csv")

In [ ]:
hm.to_csv("center_coords.csv")

# vectorize

In [ ]:
train = pd.read_csv('train_cluster_ids.csv', header=None)
test = pd.read_csv('test_cluster_ids.csv', header=None)

In [ ]:
train_and_test = pd.concat([train[1], test[1]])

In [ ]:
from sklearn.feature_extraction import DictVectorizer
vectorizer = DictVectorizer(sparse=False,dtype=np.bool)

data_dict = ( {'cluster_id':str(cluster_id)} for cluster_id in train_and_test )

train_and_test_vectorized = pd.DataFrame(vectorizer.fit_transform(data_dict), columns=vectorizer.feature_names_)

In [ ]:
train_and_test_vectorized[:len(train)].to_csv("train_clusters_vectorized.csv")
train_and_test_vectorized[len(train):].to_csv("test_clusters_vectorized.csv")